In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['페르소나 채팅']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    dialogs = {}
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            level = line['speaker']['level']['final']
            if level in ['IG']:
                continue
            interview = line['speaker']['interview']
            persona = {
                'age': line['speaker']['age'],
                'gender': line['speaker']['gender'],
                'level': level,
                '종사_분야': interview['종사_분야'],
                '현_거주_환경': interview['현_거주_환경'],
                '여가_활동_취미': interview['여가_활동_취미'],
                '영어권_거주_여부': interview['영어권_거주_여부'],
                '영어권_거주_기간': interview['영어권_거주_기간']
            }
            speaker_id = line['speaker']['id']
            question_id = line['prompt']['Question_number']
            question = line['prompt']['prompt']
            answer = line['utterance.transcription']
            qa = {'question' : question, 'answer' : answer}
            if speaker_id not in dialogs:
                dialogs[speaker_id] = {'persona': persona, 'dialog': {}}
            if question_id not in dialogs[speaker_id]['dialog']:
                dialogs[speaker_id]['dialog'][question_id] = qa
            
            #### data preprocess end 
    for speaker_id in dialogs:
        #### 페르소나 채팅
        data = {'text': {}}
        ## preprocess data from line
        data['text']['persona'] = dialogs[speaker_id]['persona']
        dialog = [v for k, v in sorted(dialogs[speaker_id]['dialog'].items())]
        data['text']['dialog'] = dialog
        ## preprocess data from line end
        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### 페르소나 채팅 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train:   0%|          | 0/1 [00:00<?, ?it/s]

valid: 100%|██████████| 1/1 [00:00<00:00, 23.65it/s]
